In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from rdkit import Chem
from rdkit.Chem import AllChem
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNetCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.decomposition import PCA

In [4]:
"""
Read in train and test as Pandas DataFrames
"""
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [9]:
df_train.iloc[1:5]

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256,gap
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.60
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.49
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.36
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.98


In [4]:
df_test.head()

,Id,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
0,1,c1sc(-c2cnc3c(c2)c2nsnc2c2cc4cccnc4cc32)c2cc[n...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,[nH]1cccc1-c1cc2c3nsnc3c3c4sccc4[nH]c3c2s1,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,[nH]1c2cc(-c3ccc[se]3)c3nsnc3c2c2c3cscc3c3ccc4...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,[nH]1c(cc2cnc3c(c12)c1=C[SiH2]C=c1c1ccc2=CCC=c...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,c1sc(-c2sc(-c3sc(-c4scc5[se]ccc45)c4ccoc34)c3c...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
#store gap values
Y_train = df_train.gap.values
#row where testing examples start
test_idx = df_train.shape[0]
#delete 'Id' column
df_test = df_test.drop(['Id'], axis=1)
#delete 'gap' column
df_train = df_train.drop(['gap'], axis=1)

In [8]:
NA_train = []
NA_test = []
GNHA_train = []
GNHA_test = []
NB_train = []
NB_test = []

for smile in df_train['smiles']:
    mol = Chem.MolFromSmiles(smile)
    NA_train.append(mol.GetNumAtoms())
    GNHA_train.append(mol.GetNumHeavyAtoms())
    NB_train.append(mol.GetNumBonds())
    
    
for smile in df_test['smiles']:
    mol = Chem.MolFromSmiles(smile)
    NA_test.append(mol.GetNumAtoms())
    GNHA_test.append(mol.GetNumHeavyAtoms())
    NB_test.append(mol.GetNumBonds())

In [9]:
df_train['NumAtms'] = NA_train
df_train['HvyAtms'] = GNHA_train
df_train['NumBnds'] = NB_train
df_test['NumAtms'] = NA_test
df_test['HvyAtms'] = GNHA_test
df_test['NumBnds'] = NB_test

In [10]:
train_smiles = df_train['smiles']
test_smiles = df_test['smiles']

In [ ]:
df_train = df_train.drop(['smiles'], axis=1)
df_test = df_test.drop(['smiles'], axis=1)

In [12]:
x_train, x_test, y_train, y_test = train_test_split(df_train, Y_train, test_size = .33)

In [13]:
molecule =Chem.MolFromSmiles(train_smiles[1])
print(molecule)

    

In [ ]:
names = ["smiles"]
for i in range(256):
    name = "feat_" + str(i)
    names.append(name)
msk = np.random.choice(np.array(train_smiles.index), size=int(len(train_smiles)/100), replace=False)
new_data=pd.DataFrame(index = [i for i in range(len(msk))], columns=names)
new_data["smiles"] = train_smiles.ix[msk].reset_index(drop=True)

new_data.head()
  

In [85]:
def feature_generator(inrow, minPathparam=1, maxPathparam=5, fpSizeparam=256, nBitsPerHashparam=1):
    molecule1 =Chem.MolFromSmiles(inrow[0])
    bitvector = Chem.rdmolops.RDKFingerprint(molecule1, minPath=minPathparam, maxPath=maxPathparam, fpSize=fpSizeparam, nBitsPerHash=nBitsPerHashparam)
    string = bitvector.ToBitString()
    string = list(string)
    data = np.array(string)
    outrow = pd.Series(data)
    return outrow

In [89]:
new_data_train = new_data.apply(feature_generator, axis=1)


    
    

In [90]:
new_data_train.insert(loc=0, column="smiles", value=train_smiles.ix[msk].reset_index(drop=True))
new_data_train.rename(mapper=(lambda x : "feat_" + str(x)), axis=1)
new_data_train.head()

,smiles,0,1,2,3,4,5,6,7,8,...,246,247,248,249,250,251,252,253,254,255
0,C1=Cc2ncc3cc4cc([se]c4cc3c2[SiH2]1)-c1scc2[SiH...,0,1,1,1,1,1,0,0,1,...,0,0,1,1,1,0,0,0,1,1
1,c1sc(-c2cc3c(o2)c2sc4ccc5cscc5c4c2c2=CCC=c32)c...,1,1,0,1,0,0,1,1,1,...,1,0,0,0,1,1,0,1,1,1
2,[nH]1c2cc(-c3cncc4nsnc34)c3=C[SiH2]C=c3c2c2oc3...,1,0,0,1,0,1,1,1,1,...,1,1,0,1,1,0,1,0,1,1
3,c1cc2cc3c(cc2[se]1)c1sc(cc1c1=C[SiH2]C=c31)-c1...,0,0,0,1,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,1
4,c1sc(-c2sc(c3Cccc23)-c2ccc(nc2)-c2ncncn2)c2occc12,0,0,1,1,0,1,1,1,0,...,1,0,1,1,0,1,0,1,1,0


In [36]:
#df_train.insert(loc=0, column="smiles", value=train_smiles)
print(msk)

AttributeError: 'numpy.ndarray' object has no attribute 'is_nan'

In [91]:
train_data_subset = df_train.ix[msk]
train_data_subset.drop(["NumAtms", "HvyAtms", "NumBnds"],axis=1, inplace=True)

train_data_subset.head()

#pd.Series(msk).reset_index(drop=True)
#df_train["smiles"]


,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
234485,C1=Cc2ncc3cc4cc([se]c4cc3c2[SiH2]1)-c1scc2[SiH...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
366231,c1sc(-c2cc3c(o2)c2sc4ccc5cscc5c4c2c2=CCC=c32)c...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
657310,[nH]1c2cc(-c3cncc4nsnc34)c3=C[SiH2]C=c3c2c2oc3...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75747,c1cc2cc3c(cc2[se]1)c1sc(cc1c1=C[SiH2]C=c31)-c1...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
204300,c1sc(-c2sc(c3Cccc23)-c2ccc(nc2)-c2ncncn2)c2occc12,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
LR = LinearRegression()
LR.fit(X_train, Y_train)
LR_pred = LR.predict(X_test)

In [ ]:
Lasso = LassoCV()
Lasso.fit(x_train, y_train)
Lasso_pred = Lasso.predict(x_test)

Lasso_error = mean_squared_error(y_test, Lasso_pred)

In [ ]:
Ridge = RidgeCV()
Ridge.fit(x_train, y_train)
Ridge_pred = Ridge.predict(x_test)

Ridge_error = mean_squared_error(y_test, Ridge_pred)

In [ ]:
l1_rtio = Lasso_error / (Lasso_error + Ridge_error)

EN = ElasticNetCV(l1_ratio = [l1_rtio, .1, .9, .95, .99, 1])
EN.fit(x_train, y_train)
EN_pred = EN.predict(x_test)

EN_error = mean_squared_error(y_test, EN_pred)

In [ ]:
Ada = AdaBoostRegressor(DecisionTreeRegressor(), learning_rate=0.05)
params = {'base_estimator__max_depth':list(range(1,6))}
ada_cv = GridSearchCV(Ada, params, cv = 5)
ada_cv.fit(x_train, y_train)

In [ ]:
# print Lasso_error
# print Ridge_error
# print EN_error
print cross_val_score(LassoCV(), df_train, Y_train)
print cross_val_score(RidgeCV(), df_train, Y_train)
print cross_val_score(ElasticNetCV(l1_ratio = l1_rtio), df_train, Y_train)

In [ ]:
cross_val_score(LinearRegression(), df_train, Y_train)

In [ ]:
LR = LinearRegression()
LR.fit(x_train, y_train)
LR_pred = LR.predict(x_test)

LR_error = mean_squared_error(y_test, LR_pred)

In [ ]:
print math.sqrt(LR_error)
print math.sqrt(Ridge_error)
print math.sqrt(Lasso_error)
print math.sqrt(EN_error)
print math.sqrt(Ada_error)

In [ ]:
#DataFrame with all train and test examples so we can more easily apply feature engineering on
df_all = pd.concat((df_train, df_test), axis=0)
df_all.head()

In [ ]:
"""
Example Feature Engineering

this calculates the length of each smile string and adds a feature column with those lengths
Note: this is NOT a good feature and will result in a lower score!
"""
#smiles_len = np.vstack(df_all.smiles.astype(str).apply(lambda x: len(x)))
#df_all['smiles_len'] = pd.DataFrame(smiles_len)


In [ ]:
#Drop the 'smiles' column
df_all = df_all.drop(['smiles'], axis=1)
vals = df_all.values
X_train = vals[:test_idx]
X_test = vals[test_idx:]
print "Train features:", X_train.shape
print "Train gap:", Y_train.shape
print "Test features:", X_test.shape

In [ ]:
LR = LinearRegression()
LR.fit(X_train, Y_train)
LR_pred = LR.predict(X_test)

In [ ]:
RF = RandomForestRegressor()
RF.fit(X_train, Y_train)
RF_pred = RF.predict(X_test)

In [ ]:
def write_to_file(filename, predictions):
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        for i,p in enumerate(predictions):
            f.write(str(i+1) + "," + str(p) + "\n")

In [ ]:
write_to_file("sample1.csv", LR_pred)
write_to_file("sample2.csv", RF_pred)

In [ ]:
print LR_pred
print RF_pred

In [ ]:
from rdkit import Chem